#### An attempt to solve the bulldozers dataset in Kaggle using Random Forests

In [80]:
#Importing requisite libraries
import pandas as pd,numpy as np,re,datetime
from IPython.display import display
from pandas.api.types import  is_string_dtype

In [ ]:
##### PARAMETERS EXPLANATION
*Setting Low_memory to prevent memory crunch in guessing the datatype

*Parse date converts string date to a date format from (11/16/2006 0:00) to (2006-11-16)

In [6]:
# Reading the data
df_raw = pd.read_csv('data/train.csv',low_memory=False ,parse_dates=['saledate'])

In [20]:
#Checking the size of the dataset and the number of columns
print (df_raw.shape)
display(df_raw.describe(include='all').transpose())

(401125, 53)
                           count unique  \
SalesID                   401125    NaN   
SalePrice                 401125    NaN   
MachineID                 401125    NaN   
ModelID                   401125    NaN   
datasource                401125    NaN   
auctioneerID              380989    NaN   
YearMade                  401125    NaN   
MachineHoursCurrentMeter  142765    NaN   
UsageBand                  69639      3   
saledate                  401125   3919   
fiModelDesc               401125   4999   
fiBaseModel               401125   1950   
fiSecondaryDesc           263934    175   
fiModelSeries              56908    122   
fiModelDescriptor          71919    139   
ProductSize               190350      6   
fiProductClassDesc        401125     74   
state                     401125     53   
ProductGroup              401125      6   
ProductGroupDesc          401125      6   
Drive_System              104361      4   
Enclosure                 400800      6  

In [34]:
#EVALUATION METRIC - RMSLE - root mean squared Log error
#Taking the log of the dependent variable
df_raw.SalePrice  = np.log(df_raw.SalePrice)                

In [70]:
#Converting the date field to various fields <saledate >
def add_datepart(df, fldname, drop=True):
    fld = df[fldname]
    if not np.issubdtype(fld.dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    for n in ('Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start'):  
        df[targ_pre+n] = getattr(fld.dt,n.lower())
    df[targ_pre+'Elapsed'] = fld.astype(np.int64) // 10**9
    if drop: df.drop(fldname, axis=1, inplace=True)


In [71]:
add_datepart(df_raw,'saledate')

KeyError: 'saledate'

In [83]:
#Converting categorical variables to numbers
def conv_to_cats(df):
    for n,c in df.items():
        if is_string_dtype(c): df[n] = c.astype('category').cat.as_ordered()

In [84]:
conv_to_cats(df_raw)